Halite Code Version 2

Basic code from YouTuber Sentdex

Added dropoff at hardcoded position depending on size of game map to reduce clogging of shipyard during mid/late game

Issues: Did not account for different positions of shipyard due to different number of players. Fixed in version 2.5. 

Having first ship move to dropoff position at beginning of game and not converting until around turn 200 is inefficient.

What is optimal number of ships?

Solutions: Spawn two new ships and move to +-8 of shipyard at around turn 200 when there is enough stored halite.

Look up on forums for ideas on ratio/number of ships.

In [ ]:
#!/usr/bin/env python3

# Import the Halite SDK, which will let you interact with the game.
import hlt
from hlt import constants
from hlt.positionals import Direction, Position
import random
import logging


# This game object contains the initial game state.
game = hlt.Game()
# Respond with your name.
game.ready("djwangBotv2")
ship_status = {}

while True:
    # Get the latest game state.
    game.update_frame()
    # You extract player metadata and the updated map metadata here for convenience.
    me = game.me
    game_map = game.game_map

    # A command queue holds all the commands you will run this turn.
    command_queue = []
    direction_order = [Direction.North, Direction.South, Direction.East, Direction.West, Direction.Still]
    positionChoices = []
    positions = []
    gameNumber = 0

    if game_map.width == 32:
        gameNumber1 = 150
        gameNumber2 = 3
        x = 8
        y = 24
    if game_map.width == 40:
        gameNumber1 = 160
        gameNumber2 = 3
        x = 11
        y = 28
    if game_map.width == 48:
        gameNumber1 = 160
        gameNumber2 = 3
        x = 12
        y = 32
    if game_map.width == 56:
        gameNumber1 = 170
        gameNumber2 = 3
        x = 14
        y = 36
    if game_map.width == 64:
        gameNumber1 = 170
        gameNumber2 = 3
        x = 16
        y = 40
    
    for ship in me.get_ships():
        if ship.id not in ship_status:
            ship_status[ship.id] = "collect"
    
        choices = ship.position.get_surrounding_cardinals() + [ship.position]
        
        positionDict = {}
        haliteDict = {}
        
        for n, direction in enumerate(direction_order):
            positionDict[direction] = choices[n]
            
        for direction in positionDict:
            position = positionDict[direction]
            haliteAmount = game_map[position].halite_amount
            if positionDict[direction] not in positionChoices:
                if direction == Direction.Still:
                    haliteDict[direction] = haliteAmount * 3
                else:
                    haliteDict[direction] = haliteAmount
    
        #logging.info("Ship {} has {} halite.".format(ship.id, ship.halite_amount))
        #logging.info("There are {} dropoffs.".format(me.get_dropoffs()))
        
        
        if game.turn_number <= 2:
            ship_status[ship.id] = "dropoff1"
        
        if ship_status[ship.id] == "dropoff1":
            mov = game_map.naive_navigate(ship, hlt.Position(x,y))
            positionChoices.append(positionDict[mov])
            command_queue.append(ship.move(mov))
            
            if mov == Direction.Still and me.halite_amount >= constants.DROPOFF_COST:
                ship_status[ship.id] = "create"
        
        elif ship_status[ship.id] == "create":
            command_queue.append(ship.make_dropoff())
        
        
        if ship_status[ship.id] == "return":
            move1 = game_map.naive_navigate(ship, me.shipyard.position)
            positionChoices.append(positionDict[move1])
            command_queue.append(ship.move(move1))
            
            if move1 == Direction.Still:
                ship_status[ship.id] = "collect"
                
        elif ship_status[ship.id] == "midGameReturn":
            sYard = game_map.calculate_distance(ship.position, me.shipyard.position)
            dOff = game_map.calculate_distance(ship.position, hlt.Position(x,y))

            if sYard <= dOff:
                move2 = game_map.naive_navigate(ship, me.shipyard.position)
                positionChoices.append(positionDict[move2])
                command_queue.append(ship.move(move2))
                
                if move2 == Direction.Still:
                    ship_status[ship.id] = "collect"
                
            elif sYard > dOff:
                move3 = game_map.naive_navigate(ship, hlt.Position(x,y))
                positionChoices.append(positionDict[move3])
                command_queue.append(ship.move(move3))
                
                if move3 == Direction.Still:
                    ship_status[ship.id] = "collect"
            
            
        elif ship_status[ship.id] == "collect":
            directionalChoice = max(haliteDict, key = haliteDict.get)
            positionChoices.append(positionDict[directionalChoice])
            command_queue.append(ship.move(game_map.naive_navigate(ship, positionDict[directionalChoice])))
            
            if ship.halite_amount > constants.MAX_HALITE * 0.95 and game.turn_number <= 200:
                ship_status[ship.id] = "return"
            elif ship.halite_amount > constants.MAX_HALITE * 0.95 and game.turn_number > 200:
                ship_status[ship.id] = "midGameReturn"

    
    if game.turn_number <= 1:
        command_queue.append(game.me.shipyard.spawn())
    
    # If you're on the first turn and have enough halite, spawn a ship.
    # Don't spawn a ship if you currently have a ship at port, though.   
    if game.turn_number <= gameNumber1 and game.turn_number >= gameNumber2 and me.halite_amount >= constants.SHIP_COST and not game_map[me.shipyard].is_occupied:
        command_queue.append(game.me.shipyard.spawn())

    # Send your moves back to the game environment, ending this turn.
    game.end_turn(command_queue)
